## U.S. EPA Pollution data analysis
Data obtained from: https://www.kaggle.com/sogun3/uspollution#

In [ ]:
#import libraries
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid")

In [ ]:
#set working directory
os.chdir('C:\\Users\\Anika\\Documents\\GradSchool\\Portfolio\\EPA_pollution')
os.getcwd() #get wd

In [ ]:
data = pd.read_csv("pollution_us_2000_2016.csv")

In [ ]:
list(data.columns) 

In [ ]:
data_top = data.head()
data_top

In [ ]:
# group by State, City, Date Local and get mean daily value of NO2 Mean, O3 Mean, SO2 Mean, CO Mean
tbl = data.groupby(['State','City','Date Local']).agg(NO2 = ('NO2 Mean', 'mean'),
                                                    O3 = ('O3 Mean', 'mean'),
                                                    SO2 = ('SO2 Mean', 'mean'),
                                                    CO = ('CO Mean', 'mean'),
                                                    count = ('NO2 Mean', 'count')).reset_index()
tbl.head()

In [ ]:
tbl.describe()

In [ ]:
tbl.info()

In [ ]:
from datetime import date
# convert the 'Date' column to datetime format
tbl['Date Local']= pd.to_datetime(tbl['Date Local'])

In [ ]:
#Extract year to a new column
tbl['Year'] = pd.DatetimeIndex(tbl['Date Local']).year

In [ ]:
print('number of cities:', tbl['City'].nunique())
print('number of rows:', tbl.shape[0])

In [ ]:
#histogram of each pollutant
fig, axes = plt.subplots(1, 4)

tbl.hist('NO2', bins=100, ax=axes[0])
tbl.hist('SO2', bins=100, ax=axes[1])
tbl.hist('O3', bins=100, ax=axes[2])
tbl.hist('CO', bins=100, ax=axes[3])

In [ ]:
n, bins, patches = plt.hist(tbl['NO2'], bins=90, facecolor='#2ab0ff', edgecolor='#e0e0e0', linewidth=0.5, alpha=0.7)

n = n.astype('int') # it MUST be integer
# Good old loop. Choose colormap of your taste
for i in range(len(patches)):
    patches[i].set_facecolor(plt.cm.viridis(n[i]/max(n)))
# Make one bin stand out   
patches[47].set_fc('red') # Set color
patches[47].set_alpha(1) # Set opacity
# Add annotation
#plt.annotate('Important Bar!', xy=(0.57, 175), xytext=(2, 130), fontsize=15, arrowprops={'width':0.4,'headwidth':7,'color':'#333333'})
# Add title and labels with custom font sizes
plt.title('Hourly mean $NO_{2}$ pollution (ppb)', fontsize=12)
plt.xlabel('$NO_{2}$ (ppb)', fontsize=10)
plt.ylabel('Values', fontsize=10)
plt.savefig('no2.png')
plt.show()

In [ ]:
#NO2 by month
no2_by_year = tbl.groupby('Year').size()
print(no2_by_year)
#Plotting the Graph
plot_by_year = no2_by_year.plot(title='Annual NO2',)
plot_by_year.set_xlabel('Years')
plot_by_year.set_ylabel('Total NO2')

In [ ]:
#Get annual max pollution value for each city then plot
tbl_year = tbl.groupby(['State','City','Year']).agg(NO2 = ('NO2', max),
                                                    O3 = ('O3', max),
                                                    SO2 = ('SO2', max),
                                                    CO = ('CO', max)).reset_index()
tbl_year.head()

## Part 1: Relationship between each pairwise set of pollutants
Note: good seaborn graphing tutorial at: https://seaborn.pydata.org/tutorial/relational.html

In [ ]:
sns.relplot(x="NO2", y="SO2", data=tbl);

In [ ]:
g = sns.relplot(x="NO2", y="SO2", hue="Year", palette="ch:r=-.5,l=.75", data=tbl_year);
g.set_axis_labels('$NO_{2}$ (ppb)','$SO_{2}$ (ppb)')
g.fig.suptitle("Relationship between $SO_{2}$ and $NO_{2}$ by year in U.S. cities")
g.savefig("so2-no2-year.png")

In [ ]:
#sns.lmplot(x="NO2", y="SO2", hue="Year", palette="ch:r=-.5,l=.75", data=tbl_year);

In [ ]:
#sns.relplot(x="NO2", y="Year", hue="SO2", palette="ch:r=-.5,l=.75", data=tbl_year);

In [ ]:
#fig, axs = plt.subplots(ncols=3)
#sns.relplot(x='NO2', y='SO2', data=tbl, ax=axs[0])
#sns.relplot(x='NO2', y='O3', data=tbl, ax=axs[1])
#sns.relplot(x='NO2', y='CO', data=tbl, ax=axs[2])

sns.relplot(x='NO2', y='SO2', data=tbl)

## Part 2: Time series for 5 worst pollution cities

In [ ]:
#get max pollution for each city
tbl_city = tbl.groupby(['State','City']).agg(NO2 = ('NO2', max),
                                            O3 = ('O3', max),
                                            SO2 = ('SO2', max),
                                            CO = ('CO', max)).reset_index()

In [ ]:
tbl_city.head()

In [ ]:
# five largest values in column 
top5_no2 = tbl_city.nlargest(5, ['NO2'])['City'].tolist()
print('NO2:',top5_no2)
top5_o3 = tbl_city.nlargest(5, ['O3'])['City'].tolist()
print('O3:',top5_o3)
top5_so2 = tbl_city.nlargest(5, ['SO2'])['City'].tolist()
print('SO2:',top5_so2)
top5_co = tbl_city.nlargest(5, ['CO'])['City'].tolist()
print('CO:',top5_co)
top5 = list(set(top5_no2) | set(top5_o3) | set(top5_so2) | set(top5_co))
print('top list:', top5)

In [ ]:
# make table with only cities in top 5 polluters for a category
tbl_top = tbl[tbl['City'].isin(top5)]
tbl_top.head()

In [ ]:
# wide to long (so pollutants are a column we can facet over)
tbl_top_long = pd.melt(tbl_top, id_vars=['State','City','Date Local','count','Year'], value_vars=['NO2','O3','SO2','CO'])
tbl_top_long.head()

In [ ]:
sns.set_theme(style="dark")

# Plot each pollutants's time series in its own facet
g = sns.relplot(
    data=tbl_top_long,
    x="Date Local", y="value", col="variable", hue="variable",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=False,
)

In [ ]:
sns.set_theme(style="dark")

# Plot each pollutants's time series in its own facet
g = sns.relplot(
    data=tbl_top_long,
    x="Date Local", y="value", col="variable", hue="variable",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=3, height=2, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    # Add the title as an annotation within the plot
    ax.text(.8, .85, year, transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=tbl_top_long, x="Date Local", y="value", units="Year",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])

# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("", "pollutant level")
g.tight_layout()